## Random Forest ##

*Crowd wisdom*, aggregated predictors usually performs better than single ones. The majority vote method is also know as *emsemble method*. The method works best if predictors are independent (data and method). Random forest is normally emsemble decision trees with bootstraping methods

## Voting Classifier ##


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

log_clf = LogisticRegression()
svm_clf = SVC()
rnd_clf = RandomForestClassifier()

voting_clf = VotingClassifier(
                estimators = [('lr', log_clf), ('svc', svm_clf), ('rf', rnd_clf)],
                voting = "hard"
            )

X, y = make_classification(n_samples= 2000, n_classes=2, n_features=20, 
                           n_redundant = 6, n_clusters_per_class = 2, 
                           weights = (0.95, 0.05))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#voting_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.953030303030303
RandomForestClassifier 0.9575757575757575
SVC 0.953030303030303
VotingClassifier 0.9545454545454546


## bagging and pasting ##
Using same training algorithm for each of predicor, but train in different subset of random data. This is called boostrap aggregating (aka bagging) if sampling is performed with replacement, pasting if without.

In [10]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, oob_score=True)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred), bag_clf.oob_score_

(0.956060606060606, 0.9529850746268657)

## Random Pathes and Random Subspace ##

Random patches is basically sampling features and data, subspace is sampling features only. Helps with high dimensional data. 

User hyperparameter: max_features and bootstrap_features

For subspace: use bootstrap = False, max_samples = 1.0


In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500, max_samples=100, bootstrap=True, oob_score=True)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
accuracy_score(y_test, y_pred), rf_clf.oob_score_

(0.9575757575757575, 0.9485074626865672)

## Extra Trees ##

*Extremely Randomised* trees. The is for every node, the trees are made random by also using random threshold for each feature, rather than search best possble threshods. This will potentially reduce bias by trading variance, and making treess faster. 

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

In [2]:
from sklearn.ensemble import ExtraTreesClassifier
ext_clf = ExtraTreesClassifier()

## Feature Importance
Random forest can also be used to measure the relative imporance of each feature, by measure how much reduction of impurity for each node using weighted average.

In [7]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09726536148450997
sepal width (cm) 0.027636121658226105
petal length (cm) 0.42961885119732585
petal width (cm) 0.44547966565993813


## Boosting
Boosting involves combine several weak learners. This is to train predictors sequentially, each trying to correct its predecessor.

Popular methods are **AdaBoost**(adaptive boosting) and **Gradient Boost**

https://www.sciencedirect.com/science/article/pii/S002200009791504X


### ADA Boosting ###
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
First base predictor is trained, relative weights of misclassified instances is then increased, the next predictor is trained using updated weights.

https://towardsdatascience.com/boosting-and-adaboost-clearly-explained-856e21152d3e

Start by assigning a weight for each sample, then increase weight for misclassified samples, find the best stump for the next one, and start again.

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

iris = load_iris()
ada_clf = AdaBoostClassifier(n_estimators=50)
ada_clf.fit(iris['data'], iris['target'])

s = cross_val_score(ada_clf, iris['data'], iris['target'], cv=6)
print(s.mean())

ada_clf.estimator_errors_, ada_clf.estimator_weights_, ada_clf.feature_importances_, iris['feature_names']


0.9466666666666667


(array([3.33333333e-01, 6.00022589e-02, 3.33355372e-01, 1.20397414e-01,
        3.33354186e-01, 2.46496772e-04, 3.33379977e-01, 3.24696290e-04,
        3.33365707e-01, 3.77301322e-04, 3.33379704e-01, 3.49084999e-04,
        3.33368069e-01, 3.77542089e-04, 3.33377933e-01, 3.52106376e-04,
        3.33369433e-01, 3.72712090e-04, 3.33376616e-01, 3.54056001e-04,
        3.33370427e-01, 3.69265052e-04, 3.33375674e-01, 3.55694330e-04,
        3.33371161e-01, 3.66896338e-04, 3.33374996e-01, 3.57017448e-04,
        3.33371703e-01, 3.65249478e-04, 3.33374507e-01, 3.58050974e-04,
        3.33372101e-01, 3.64091057e-04, 3.33374151e-01, 3.58841485e-04,
        3.33372394e-01, 3.63268582e-04, 3.33373893e-01, 3.59437902e-04,
        3.33372609e-01, 3.62680377e-04, 3.33373705e-01, 3.59883751e-04,
        3.33372767e-01, 3.62257368e-04, 3.33373568e-01, 3.60214928e-04,
        3.33372882e-01, 3.61951873e-04]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.